# Test Vehicle Tracking


In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import os
import pickle
from moviepy.editor import VideoFileClip
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sdc_functions import *
from scipy.ndimage.measurements import label
from sklearn.model_selection import train_test_split
from config import Config
from tqdm import tqdm

%matplotlib inline

vehicles_folder = '../vehicles'
non_vehicles_folder = '../non-vehicles'

vehicles = glob.glob(vehicles_folder + '/**/*.png', recursive=True)
print('vehicles = ', len(vehicles))
print('image0 = ', vehicles[0])

non_vehicles = glob.glob(non_vehicles_folder + '/**/*.png', recursive=True)
print('non_vehicles = ', len(non_vehicles))
print('image0 = ', non_vehicles[0])

### Subsample 
sample_size = 500
vehicles = np.random.choice(vehicles, sample_size)
non_vehicles = np.random.choice(non_vehicles, sample_size)
print('Sampled to %d images.' % sample_size)

### Parameters
# color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
# orient = 18  # HOG orientations
# pix_per_cell = 6 # HOG pixels per cell
# cell_per_block = 2 # HOG cells per block
# hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
# spatial_size = (24, 24) # Spatial binning dimensions
# hist_bins = 16    # Number of histogram bins
# spatial_feat = True # Spatial features on or off
# hist_feat = True # Histogram features on or off
# hog_feat = True # HOG features on or off
# y_start_stop = [300, 720] # Min and max in y to search in slide_window()

config = Config()

color_space = config.color_space # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = config.orient  # HOG orientations
pix_per_cell = config.pix_per_cell # HOG pixels per cell
cell_per_block = config.cell_per_block # HOG cells per block
hog_channel = config.hog_channel # Can be 0, 1, 2, or "ALL"
spatial_size = config.spatial_size # Spatial binning dimensions
hist_bins = config.hist_bins    # Number of histogram bins
spatial_feat = config.spatial_feat # Spatial features on or off
hist_feat = config.hist_feat # Histogram features on or off
hog_feat = config.hog_feat # HOG features on or off
y_start_stop = config.y_start_stop # Min and max in y to search in slide_window()



vehicles =  8792
image0 =  ../vehicles/GTI_Far/image0000.png
non_vehicles =  8968
image0 =  ../non-vehicles/Extras/extra1.png
Sampled to 500 images.


In [ ]:
## Train SVC

t=time.time()
car_features = extract_features(vehicles, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(non_vehicles, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
t2=time.time()

print('Feature extracted :')
print('scaled_X =', len(scaled_X))
print('y =', len(y))
print('feature length = ', len(scaled_X[0]))
print('Time %.2f s' % (t2-t))



### Train SVC

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.01, random_state=rand_state)

print('X_train =', len(X_train));
print('X_test =', len(X_test));
print('y_train =', len(y_train));
print('y_test =', len(y_test));

# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test)*100, 4))

## Store to file
store_data = {"svc":svc, "scaler": X_scaler}
with open('pickle_svc_scaler.p', 'wb') as f:
    pickle.dump(store_data, f)

## Restore
restore_data = pickle.load(open('pickle_svc_scaler.p', "rb" ))
svc = restore_data["svc"]
X_scaler = restore_data["scaler"]

## Draw test prediction
# plt.figure(figsize=(20, 20))
f, axarr = plt.subplots(10, 10, figsize=(20, 20))
for r in range(len(axarr)):
    for c in range(len(axarr[0])):
        if np.random.uniform() < 0.5:
            im = imread(np.random.choice(vehicles))
        else:
            im = imread(np.random.choice(non_vehicles))
        im = cv2.resize(im, (64, 64))
        features = extract_features_img(im, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        test_features = X_scaler.transform(np.array(features).reshape(1, -1))
        decision = svc.decision_function(test_features)
        axarr[r, c].axes.get_xaxis().set_ticks([])
        axarr[r, c].axes.get_yaxis().set_ticks([])
        axarr[r, c].set_title("%.2f" % decision[0], fontsize=12)
        axarr[r, c].imshow(im)



In [5]:
## Search in windows

# from sdc_functions import *

## Restore
restore_data = pickle.load(open('pickle_svc_scaler.p', "rb" ))
svc = restore_data["svc"]
X_scaler = restore_data["scaler"]


test_images = []
# test_images = ['test_images/test1.jpg',
#               'test_images/test2.jpg',
#               'test_images/test3.jpg',
#               'test_images/test4.jpg',
#               'test_images/test5.jpg',
#               'test_images/test6.jpg']

# images = [imread(img) for img in test_images]

window_sizes = [160, 128, 96] # [256, 224, 192, 160, 128, 96, 64]
window_sizes_box = [(b, b) for b in window_sizes]
# print('window_sizes_box =', window_sizes_box)


overlap_factor = 0.70
all_windows = []


car_bboxes = []

def process_func(image):
    global car_bboxes
    global all_windows
    resImg = np.zeros_like(image)
    
    # Show: Original Image
    compose_images(resImg, image, 2, 2, 3)
    
    if len(all_windows) == 0:
        # Calculate all_windows positions (only once)
        t0 = time.time()
        for widx, window_size in enumerate(window_sizes_box):
            windows = slide_window(image.shape, x_start_stop=[None, None], y_start_stop=config.y_start_stop, 
                            xy_window=window_size, xy_overlap=(overlap_factor, overlap_factor))
            all_windows.extend(windows)

        print('all_windows len = %d, Time: %.4f seconds' % (len(all_windows), (time.time() - t0)))


    t1 = time.time()
    
    all_hot_windows = []
    all_hot_windows = search_windows(image, all_windows, svc, X_scaler, color_space=config.color_space, 
                                spatial_size=config.spatial_size, hist_bins=config.hist_bins, 
                                orient=config.orient, pix_per_cell=config.pix_per_cell, 
                                cell_per_block=config.cell_per_block, 
                                hog_channel=config.hog_channel, spatial_feat=config.spatial_feat, 
                                hist_feat=config.hist_feat, hog_feat=config.hog_feat)
    t2 = time.time()
#     print('Hot Windows Time: %.4f' % (t2-t1))
    
    '''
    for widx, window_size in enumerate(window_sizes_box):
        t = time.time()
#         overlap_factor = 1. - max(0.3, 20.0/window_size[0])
        overlap_factor = 0.7
        windows = slide_window(image, x_start_stop=[None, None], y_start_stop=config.y_start_stop, 
                        xy_window=window_size, xy_overlap=(overlap_factor, overlap_factor))
#         all_windows.append(windows)
        t2 = time.time()
        
#         print('window_size=',window_size,'overlap=',overlap_factor,' num=', len(windows), ', %.2f seconds' % (t2-t) )

#     print('windows len =', len(windows))
#     print('windows[0] =', windows[0])

        hot_windows = search_windows(image, windows, svc, X_scaler, color_space=config.color_space, 
                                spatial_size=config.spatial_size, hist_bins=config.hist_bins, 
                                orient=config.orient, pix_per_cell=config.pix_per_cell, 
                                cell_per_block=config.cell_per_block, 
                                hog_channel=config.hog_channel, spatial_feat=config.spatial_feat, 
                                hist_feat=config.hist_feat, hog_feat=config.hog_feat)
        t3 = time.time()
#         print('hog time %.2f seconds' % (t3-t2) )
    
#         windows_counts[widx] += len(hot_windows)
        
    
        all_hot_windows.extend(hot_windows)
        
#     print('windows_counts =', windows_counts)
    '''


    ## Draw All Boxes
    draw_image = np.copy(image)
    windows_img = draw_boxes(draw_image, all_hot_windows, color=(0, 0, 255), thick=2)
    compose_images(resImg, windows_img, 2, 2, 1)
    t3 = time.time()
#     print('Draw All Boxes Time: %.4f' % (t3 - t2))
    
    ## Draw ALL ALL Boxes
#     draw_image = np.copy(image)
#     window_img = draw_boxes(draw_image, all_windows[0], color=(255, 0, 0), thick=2)                    
#     axs[idx, 1].imshow(window_img)

    
    ## Heat Map
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    heat = add_heat(heat, all_hot_windows)
    
    ## Draw Heat Map 1
    final_map = np.clip(heat, 0, 255)
    fig = plt.figure(figsize=(10, 5))
    plt.title('Start')
    plt.imshow(final_map, cmap='hot')
    data = get_fig_image(fig)
    compose_images(resImg, data, 4, 4, 3)

    
    # Add previous frame cars
    heat = add_heat(heat, car_bboxes, scale=2)
    
    ## Draw Heat Map 2
    final_map = np.clip(heat, 0, 255)
    fig = plt.figure(figsize=(10, 5))
    plt.title('With Prev Car')
    plt.imshow(final_map, cmap='hot')
    data = get_fig_image(fig)
    compose_images(resImg, data, 4, 4, 4)


    heat = apply_threshold(heat, 2)
    
    ## Draw Heat Map 3
    final_map = np.clip(heat, 0, 255)
    fig = plt.figure(figsize=(10, 5))
    plt.title('Thresholded')
    plt.imshow(final_map, cmap='hot')
    data = get_fig_image(fig)
    compose_images(resImg, data, 4, 4, 7)

    
    labels = label(heat)
    t4 = time.time()
#     print('HeatMap, Label Time: %.4f' % (t4 - t3))

    
    ## Draw Label Map
    fig = plt.figure(figsize=(10, 5))
    plt.title('%d cars found' % labels[1])
    plt.imshow(labels[0], cmap='gray')
    data = get_fig_image(fig)
    compose_images(resImg, data, 4, 4, 8)


#     print(labels[1], 'cars found')
#     plt.subplot(1, 2, 2)

    ## Dtaw Labels Map
#     axs[idx, 2].set_title('%d cars found' % labels[1])
#     axs[idx, 2].imshow(labels[0], cmap='gray')
    
    ## Draw Outer Box
    bbox_image, car_bboxes = draw_labeled_bboxes(image, labels, all_hot_windows)
    compose_images(resImg, bbox_image, 2, 2, 4)
    t5 = time.time()
#     print('Draw Outer Time: %.4f' % (t5 - t4))

    
    
    return resImg

# Test Images
for idx, timg in enumerate(tqdm(test_images)):
    image = imread(timg)

    draw_image = process_func(image)
    plt.figure(figsize=(10, 5))
    plt.imshow(draw_image)
    

# Process Video
video_file = 'project_video.mp4'
output_video_file = 'output.mp4'
t_start = 15.0
t_end = 15.2
clip = VideoFileClip(video_file)
if t_end > 0.0:
    clip = clip.subclip(t_start=t_start, t_end=t_end)
else:
    clip = clip.subclip(t_start=t_start)
clip = clip.fl_image(process_func)
clip.write_videofile(output_video_file, audio=False)
    


'''
## Draw samples from all_hot_windows
# plt.figure(figsize=(20, 20))
f, axarr = plt.subplots(10, 10, figsize=(20, 20))
for r in range(len(axarr)):
    for c in range(len(axarr[0])):
        window = all_hot_windows[np.random.randint(0, len(all_hot_windows))]
        im = cv2.resize(image[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        features = extract_features_img(im, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        test_features = X_scaler.transform(np.array(features).reshape(1, -1))
        decision = svc.decision_function(test_features)
        axarr[r, c].axes.get_xaxis().set_ticks([])
        axarr[r, c].axes.get_yaxis().set_ticks([])
        axarr[r, c].set_title("%.2f" % decision[0], fontsize=12)
        axarr[r, c].imshow(im)
        
        
## Draw test prediction
# plt.figure(figsize=(20, 20))
f, axarr = plt.subplots(5, 5, figsize=(20, 20))
for r in range(len(axarr)):
    for c in range(len(axarr[0])):
        if np.random.uniform() < 0.5:
            im = imread(np.random.choice(vehicles))
        else:
            im = imread(np.random.choice(non_vehicles))
        im = cv2.resize(im, (64, 64))
        features = extract_features_img(im, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        test_features = X_scaler.transform(np.array(features).reshape(1, -1))
        decision = svc.decision_function(test_features)
        axarr[r, c].axes.get_xaxis().set_ticks([])
        axarr[r, c].axes.get_yaxis().set_ticks([])
        axarr[r, c].set_title("%.2f" % decision[0], fontsize=12)
        axarr[r, c].imshow(im)

'''





0it [00:00, ?it/s]


all_windows len = 1111, Time: 0.0018 seconds
[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|██████████| 5/5 [00:36<00:00,  7.32s/it]

[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 



'\n## Draw samples from all_hot_windows\n# plt.figure(figsize=(20, 20))\nf, axarr = plt.subplots(10, 10, figsize=(20, 20))\nfor r in range(len(axarr)):\n    for c in range(len(axarr[0])):\n        window = all_hot_windows[np.random.randint(0, len(all_hot_windows))]\n        im = cv2.resize(image[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))\n        features = extract_features_img(im, color_space=color_space, \n                            spatial_size=spatial_size, hist_bins=hist_bins, \n                            orient=orient, pix_per_cell=pix_per_cell, \n                            cell_per_block=cell_per_block, \n                            hog_channel=hog_channel, spatial_feat=spatial_feat, \n                            hist_feat=hist_feat, hog_feat=hog_feat)\n        test_features = X_scaler.transform(np.array(features).reshape(1, -1))\n        decision = svc.decision_function(test_features)\n        axarr[r, c].axes.get_xaxis().set_ticks([])\n        axarr[r,

In [ ]:
## Draw ALL ALL Boxes

# video_file = 'project_video.mp4'
# output_video_file = 'output.mp4'
# t_start = 4.0
# t_end = 14.0
# clip = VideoFileClip(video_file)
# if t_end > 0.0:
#     clip = clip.subclip(t_start=t_start, t_end=t_end)
# else:
#     clip = clip.subclip(t_start=t_start)
# clip = clip.fl_image(process_func)
# clip.write_videofile(output_video_file, audio=False)


In [6]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video_file))

In [34]:
def find_outer_box(box, hot_windows):
  x1 = box[0][0]
  x2 = box[1][0]
  y1 = box[0][1]
  y2 = box[1][1]
  minX = x1
  maxX = x2
  minY = y1
  maxY = y2
  print('box =', box)

  for window in hot_windows:
    print('window =', window)
    xOutside = (window[0][0] > x2 and window[1][0] > x2) or (window[0][0] < x1 and window[1][0] < x1)
    yOutside = (window[0][1] > y2 and window[1][1] > y2) or (window[0][1] < y1 and window[1][1] < y1)
    print('xout =', xOutside, 'yout =', yOutside)
    if not (xOutside or yOutside):
      # Match
      if window[0][0] < minX:
        minX = window[0][0]
      if window[0][1] < minY:
        minY = window[0][1]
      if window[1][0] > maxX:
        maxX = window[1][0]
      if window[1][1] > maxY:
        maxY = window[1][1]
      print('match: ', ((minX, minY), (maxX, maxY)))
  return ((minX, minY), (maxX, maxY))

box = ((10,10),(15,15))
hot_windows = [
#     ((0,0),(20,20)),
#     ((5,5),(18,18)),
    ((5,5),(12,12)),
    ((14,14),(22,50))
]

# res = find_outer_box(box, hot_windows)


def intersection(box1, box2):
    x = max(box1[0][0], box2[0][0])
    y = max(box1[0][1], box2[0][1])
    w = min(box1[1][0], box2[1][0]) - x
    h = min(box1[1][1], box2[1][1]) - y
    if w < 0 or h < 0: return 0
    return w*h

def combine_boxes(prev_box, new_box, prev_factor = 0.9):
    print('prev_box =', prev_box)
    print('new_box =', new_box)
    minX = int(round(prev_box[0][0] * prev_factor + new_box[0][0] * (1. - prev_factor)))
    minY = int(round(prev_box[0][1] * prev_factor + new_box[0][1] * (1. - prev_factor)))
    maxX = int(round(prev_box[1][0] * prev_factor + new_box[1][0] * (1. - prev_factor)))
    maxY = int(round(prev_box[1][1] * prev_factor + new_box[1][1] * (1. - prev_factor)))
    print('comb_res =', ((minX, minY),(maxX, maxY)))
    return ((minX, minY),(maxX, maxY))

def find_max_intersection(cars_list, box):
    max_v = 0
    max_ind = -1
    # For car
    for ind, car_box in enumerate(cars_list):
        inter = intersection(car_box, box)
        if inter > max_v:
            max_v = inter
            max_ind = ind
    return max_v, max_ind

def combine_with_prev(prev, prev_age, curr, prev_factor=0.8):
    new_cars = []
    new_cars_age = []
    prev_cars = prev[:]
    prev_cars_age = prev_age[:]
    curr_cars = curr[:]
    fresh_car_age = 2
    
    while len(prev_cars) > 0:
        prev_car = prev_cars.pop()
        prev_car_age = prev_cars_age.pop()
        inter_v, inter_ind = find_max_intersection(curr_cars, prev_car)
        if inter_v > 0:
            # Found car in a curr list
            new_cars.append(combine_boxes(prev_car, curr_cars[inter_ind], prev_factor=prev_factor))
            new_cars_age.append(fresh_car_age)
            del curr_cars[inter_ind]
        else:
            # Not found in curr list, check for age
            if prev_car_age > 0:
                # Add and decrease age
                new_cars.append(prev_car)
                new_cars_age.append(prev_car_age - 1)
                
    # Add all other current cars to the list of new cars
    while len(curr_cars) > 0:
        curr_car = curr_cars.pop()
        new_cars.append(curr_car)
        new_cars_age.append(fresh_car_age)
        
    return new_cars, new_cars_age
    

prev_cars = [
    ((10,10),(20,20)),
    ((30,30),(40,40))
]

prev_cars_age = [
    2,
    0
]

curr_cars = [
    ((18,18),(28,28)),
    ((44,44),(54,54)),
    ((80,80),(90,90))
]

  
image = np.zeros((720, 1280))

def get_window_around(image, point, window_size):
    maxX, maxY = image.shape[1] - 1, image.shape[0] - 1
    x1 = point[0] - window_size//2
    x2 = point[0] + window_size//2
    y1 = point[1] - window_size//2
    y2 = point[1] + window_size//2
    # Clip
    if x1 < 0: x1 = 0
    if x2 > maxX: x2 = maxX
    if y1 < 0: y1 = 0
    if y2 > maxY: y2 = maxY
    return ((x1,y1),(x2,y2))
    

def get_detailed_windows(image, car_box, window_sizes):
    # print('cw =', cw)
    cX = (car_box[1][0] + car_box[0][0])//2
    cY = (car_box[1][1] + car_box[0][1])//2
    # print('cX =', cX, ' cY=', cY)
    
    # Get the half of the smallest window
    delta = int(window_sizes[-1] * 0.5)
    
    # Shift window to delta in each direction
    deltak = [
        (-1,-1), (0,-1), (1,-1),
        (-1, 0), (0, 0), (1, 0),
        (-1, 1), (0, 1), (1, 1)
    ]
    
    # Iterate over all window_sizes and all positions
    windows = []
    for wind in window_sizes:
        for k in deltak:
            kx, ky = k[0], k[1]
            cx = int(cX + kx * delta)
            cy = int(cY + ky * delta)
            w = get_window_around(image, (cx, cy), wind)
            windows.append(w)
      # print('app ', ((x1,y1),(x2,y2)))
    return windows
      

    
    
def cars_search_windows(image, prev_cars):
  maxX, maxY = image.shape[1] - 1, image.shape[0] - 1
  cars_windows = []
  window_sizes = [256, 224, 192, 160, 128, 96] # [256, 224, 192, 160, 128, 96, 64]
  for cw in prev_cars:
    wins = get_detailed_windows(image, cw, window_sizes)
    cars_windows.extend(wins)
  return cars_windows
    

res = cars_search_windows(image, curr_cars)
    
# res = combine_with_prev(prev_cars, prev_cars_age, curr_cars, 0.5)

print('res len =', len(res))
print('res =', res)

res len = 162
res = [((0, 0), (103, 103)), ((0, 0), (151, 103)), ((0, 0), (199, 103)), ((0, 0), (103, 151)), ((0, 0), (151, 151)), ((0, 0), (199, 151)), ((0, 0), (103, 199)), ((0, 0), (151, 199)), ((0, 0), (199, 199)), ((0, 0), (87, 87)), ((0, 0), (135, 87)), ((0, 0), (183, 87)), ((0, 0), (87, 135)), ((0, 0), (135, 135)), ((0, 0), (183, 135)), ((0, 0), (87, 183)), ((0, 0), (135, 183)), ((0, 0), (183, 183)), ((0, 0), (71, 71)), ((0, 0), (119, 71)), ((0, 0), (167, 71)), ((0, 0), (71, 119)), ((0, 0), (119, 119)), ((0, 0), (167, 119)), ((0, 0), (71, 167)), ((0, 0), (119, 167)), ((0, 0), (167, 167)), ((0, 0), (55, 55)), ((0, 0), (103, 55)), ((0, 0), (151, 55)), ((0, 0), (55, 103)), ((0, 0), (103, 103)), ((0, 0), (151, 103)), ((0, 0), (55, 151)), ((0, 0), (103, 151)), ((0, 0), (151, 151)), ((0, 0), (39, 39)), ((0, 0), (87, 39)), ((7, 0), (135, 39)), ((0, 0), (39, 87)), ((0, 0), (87, 87)), ((7, 0), (135, 87)), ((0, 7), (39, 135)), ((0, 7), (87, 135)), ((7, 7), (135, 135)), ((0, 0), (23, 23)),